<a href="https://colab.research.google.com/github/DMXMax/covid19/blob/gc_20200315/coviddata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests,json
import datetime


In [ ]:
covidtrackingurl = "https://covidtracking.com/api/v1/states/daily.json"
response = requests.get(covidtrackingurl)
dfCovidtracking = pd.DataFrame(response.json())
dfCovidtracking.fillna(0, inplace=True)
dfCovidtracking.negative = dfCovidtracking.negative.astype('int64')
dfCovidtracking.positive = dfCovidtracking.positive.astype('int64')
dfCovidtracking.pending = dfCovidtracking.pending.astype('int64')
dfCovidtracking.total = dfCovidtracking.total.astype('int64')
dfCovidtracking.death = dfCovidtracking.death.astype('int64')

dfCovidtracking.date = dfCovidtracking.date.transform(lambda x: pd.to_datetime(datetime.date(x//10000, x%10000//100, x%100
                                          )))
dfCovidtracking.set_index(['date'], inplace=True)


In [ ]:
dfWHO = pd.read_csv("https://raw.githubusercontent.com/DMXMax/covid19/master/who/who_data.tsv",
                     sep='\t', index_col='Date')

In [ ]:
jhurl = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/"+\
"csse_covid_19_data/csse_covid_19_daily_reports/06-21-2020.csv"


dfJHUData = pd.read_csv(jhurl, header=0, usecols=[0,1,2,3,4,5,6,7,8,9,10,11],
                        names=['FIPS', 'city','state', 'country', 'last_update',
                               'lat', 'lon','confirmed', 'dead', 'recovered','active', 'key'])

In [ ]:
dfWHO = dfWHO.rename(columns={"Confirmed": "confirmed", "Deaths": "who_deaths"})
dfWHO.index.names=['date']


In [ ]:
print("US Summary")
dfCovidtracking.groupby(by='date').sum()[-14:][['positive', 'death', 'positiveIncrease', 'deathIncrease']]

In [ ]:
print(dfCovidtracking.groupby(by='date').sum()[-21:].positiveIncrease.rolling(window=7).mean())
print(dfCovidtracking.groupby(by='date').sum()[-21:].positiveIncrease.std())
print(dfCovidtracking.groupby(by='date').sum()[-21:].deathIncrease.rolling(window=1).mean())
print(dfCovidtracking.groupby(by='date').sum()[-21:].deathIncrease.std())
dta = dfCovidtracking.groupby(by='date').sum()[-1:]
print(f"Testing hit score: {(dta.positive/dta.totalTestResults)[0]:2.3f}")
print(f"IFR hit score: {(dta.death/dta.positive)[0]:2.3f}")

In [ ]:
dfCovidtracking.loc[dfCovidtracking.state=='FL'].groupby(by='date').sum()[['positive', 'death', 'positiveIncrease', 'deathIncrease']].tail()

In [ ]:
dfnormalized = dfCovidtracking.groupby(by='date').sum()[['positiveIncrease', 'deathIncrease']].copy()


In [ ]:
((dfnormalized-dfnormalized.min())/(dfnormalized.max()-dfnormalized.min())).plot.line(figsize=(14,7))

In [ ]:
dfctd_state_curr = dfCovidtracking.loc[dfCovidtracking.index.max()] \
  .rename(columns={'positive':'total_positive'}) \
  .set_index('state') 

print(f"Total positive cases = {dfctd_state_curr.total_positive.sum()}")
print(f"Total deaths = {dfctd_state_curr.death.sum()}")
  
dfctd_state_curr.sort_values(by='total_positive', ascending=False).head(5)

In [ ]:
curr_state = 'CA'
gbUSA = dfCovidtracking.groupby(['date'])
gbState = dfCovidtracking[dfCovidtracking.state==curr_state].groupby(['date'])

cols = ['positive', 'totalTestResultsIncrease', 'death', 'positiveIncrease','deathIncrease']

dfUSAPos = gbUSA[cols].sum().reset_index().copy()
dfStatePos = gbState[cols].sum().reset_index().copy()

#This will calculate the percentage each day is of the total. In a flattening, this will be more even
# In a high growth situation, you'll higher percentages at the tail. 

dfUSAPos['pos_pct']= dfUSAPos.positive/dfctd_state_curr.total_positive.sum()
dfStatePos['pos_pct']=dfStatePos.positive/dfctd_state_curr.loc[curr_state].total_positive

In [ ]:
dfUSAPos[-7:]

In [ ]:
dfStatePos[-7:]

In [ ]:
gbUSA.sum()[-5:]

In [ ]:
gbState.sum()[-5:]

In [ ]:
x=gbUSA.sum().loc[:, ['positive', 'negative','death','total']].plot.line()

In [ ]:
dfResults = gbUSA.sum().join(dfWHO, how='left', rsuffix='_who' )
dfResults.loc[:, ['positive', 'confirmed']].plot.line()

# John Hopkins Data


In [ ]:
dfData = dfJHUData[dfJHUData.country=='US'][['confirmed', 'dead']].sum()
print(dfData)
print(f"IFR based on JHU Data: {dfData.dead/dfData.confirmed*100:.2f}%")

In [ ]:
dfJHUData[(dfJHUData.country=='US')&(dfJHUData.state=='California')][['confirmed', 'dead']].sum()

In [ ]:
dfJHUUS = dfJHUData[dfJHUData.country=='US']
gbJHUByState= dfJHUUS.groupby(by='state')
gbJHUByCountry = dfJHUData.groupby(by='country')

In [ ]:
dfJHUData.head()

In [ ]:
dfJHUData[dfJHUData.city=="Marin"]

In [ ]:
gbJHUByCountry.confirmed.sum().sort_values(ascending=False).head(10)

In [ ]:
dfctd_state_curr.head()

In [ ]:
dfState = dfCovidtracking.reset_index()[['date','state','positive', 'death']].set_index('state') \
  .join(dfctd_state_curr.total_positive)

In [ ]:
dfState['pct']=dfState.positive/dfState.total_positive

In [ ]:
dfState.loc['FL'].head(14)


In [ ]:
res = dfState.reset_index().pivot(index = 'date', columns='state', values='pct').fillna(0)

In [ ]:
x = res.loc[:, ['CA','FL']].plot.line()

In [ ]:
x = dfCovidtracking.groupby(by='date')
y1=x.sum()[-33:].deathIncrease.plot.line(figsize=(40,20), fontsize=36, \
  style='.-', markersize=48, color='red', linewidth=4) \
.legend(['Daily Additional Deaths'],fontsize=32)


In [ ]:
x = dfCovidtracking.groupby(by='date')
y1=x.sum()

chart = y1[-90:].positiveIncrease.plot.line(figsize=(40,20), fontsize=36, \
  style='.-', markersize=48, color='red', linewidth=4) \
.legend(['Daily Additional Positive Tests'],fontsize=32)


In [ ]:
def getWeek(dt):
  return (dt.year*100)+dt.week

dfByWeekYear = dfCovidtracking[['deathIncrease', 'hospitalizedIncrease', 
                                'negativeIncrease','positiveIncrease','totalTestResultsIncrease']] \
                                .groupby(by=getWeek)


In [ ]:
def getWeek(dt):
  return (dt.year*100)+dt.week

dfDoW = dfCovidtracking[['deathIncrease', 'hospitalizedIncrease', 
                         'negativeIncrease','positiveIncrease','totalTestResultsIncrease']] \
                         .groupby(by=lambda x: x.dayofweek)


In [ ]:
dfDoW.mean().deathIncrease.plot.bar()



In [ ]:
dfByWeekYear.sum()[-7:]

In [ ]:
import matplotlib.ticker as ticker
plt1 = dfByWeekYear.sum()[:-1][['deathIncrease']].plot.line(figsize=(40,20), \
    fontsize=36, style='.-', markersize=48, linewidth=10, color='red')
plt1.legend(['Weekly Additional Deaths'],fontsize=32)
plt2 = dfByWeekYear.sum()[:-1][['totalTestResultsIncrease']].plot.line(figsize=(40,20), \
    fontsize=36, style='.-', markersize=48, linewidth=10, color='purple')
plt2.legend(['Weekly Additional Tests'],fontsize=32)

plt2.yaxis.set_major_formatter(ticker.FormatStrFormatter('%6.0d'))
    
plt3 = dfByWeekYear.sum()[:-1][['positiveIncrease']].plot.line(figsize=(40,20), \
    fontsize=36, style='.-', markersize=48, linewidth=10, color='green')
plt3.legend(['Weekly Additional Postives'],fontsize=32)

In [ ]:
dfCovidtracking.groupby(by='date').sum().rolling(window=7).mean().positiveIncrease.plot.line()

In [ ]:
dfCovidtracking.groupby(by='date').sum().rolling(window=7).mean().deathIncrease.plot.line()

In [ ]:
dfCAAvg = dfCovidtracking[dfCovidtracking.state=='TX'] \
.groupby(by='date').sum().rolling(window=7).mean()

In [ ]:

graph1 = dfCAAvg[-40:].positiveIncrease.plot.bar(figsize=(14,7))


In [ ]:
dfCAAvg.deathIncrease.plot.line()

In [ ]:
dfDates = dfState.reset_index().copy().set_index('date')
dfDates[(dfDates.state.isin(['AZ']))].head(10)[::-1].positive.diff()


In [ ]:
datecutoff = dfDates.index.max()-datetime.timedelta(days=7)
dfDates[(dfDates.index == datecutoff) & (dfDates.pct < .85)].sort_values(by='pct')


In [ ]:

def showStateData(state):
  return dfCovidtracking[dfCovidtracking.state==state]\
  .sort_index()\
  .copy()



chart = showStateData('MI')\
.positiveIncrease\
.rolling(window=7)\
.mean()[-60:].plot.bar(figsize=(14,5))

In [ ]:
dfTest = dfCovidtracking[dfCovidtracking.state=='CA'].sort_index().copy()
dfTest.head()

In [ ]:
dfTest.head()

# State Population Data


In [ ]:
data_url = "https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-alldata.csv"


In [ ]:
dfUSPop = pd.read_csv(data_url)

In [ ]:
import math
us_pop = dfUSPop.loc[0].POPESTIMATE2019 #Estimate from Wikipedia
res = dfCovidtracking.loc[dfCovidtracking.index.max()][['positive', 'negative', 'pending','death', 'total']].sum()
cir = res.positive/res.total
cfr = res.death/res.positive
p_aff= math.ceil(us_pop*cir)
p_dead= math.ceil(p_aff*cfr)
print(res)
print(f"Current Infection Rate {cir*100:2f}%")
print(f"Percentage of US affected: {(res.positive+res.death)/us_pop*100:2f}%")
print(f"Observed CFR: {cfr*100:2f}%")
print(f"Percentage of US tested/affected: {(res.negative+res.positive+res.death)/us_pop*100:2f}%")
print(f"Projected affected by cir = {p_aff:,d}")
print(f"Projected killed = {p_dead:,d}")


In [ ]:
!if [ ! -d "loc" ]; then \
git clone https://gist.github.com/rogerallen/1583593 --depth=1 --quiet loc; \
fi
import sys
sys.path.append("loc")




In [ ]:
import us_state_abbrev as us_abbr


In [ ]:
dfStatePop = dfUSPop[dfUSPop.STATE != 0].copy()

In [ ]:
dfStatePop['ABBR']=dfStatePop.NAME.apply(lambda x: us_abbr.us_state_abbrev[x])

In [ ]:
dfStatePop = dfStatePop.loc[:, ['ABBR', 'POPESTIMATE2019']]
dfStatePop.set_index('ABBR', inplace=True)

In [ ]:
dfStateCovid = dfCovidtracking.loc[dfCovidtracking.index.max()].set_index('state')

In [ ]:
dfStateJoin = dfStateCovid.join(dfStatePop)

In [ ]:
r1 = pd.DataFrame([(dfStateJoin.positive / dfStateJoin.POPESTIMATE2019)*100, \
              (dfStateJoin.totalTestResults/ dfStateJoin.POPESTIMATE2019)*100],\
             index=['state_pos_pop','state_test_pop'] ).\
             T.sort_values(by="state_pos_pop", ascending=False)[:50]

print(r1.loc[['CA', 'FL','MN','TX', 'MI','OK']])
print(r1[:10])

In [ ]:
#print(f"Current Infection Rate {res.positive/res.total*100:2f}%")

print(f"Current USA Testing Percent {gbUSA.sum().tail(1).totalTestResults[0]/dfUSPop[dfUSPop.REGION=='0'].POPESTIMATE2019[0]*100:2f}%")

# OWID Data

In [ ]:
dfOwid = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv")

In [ ]:
dfOwidTests = dfOwid.dropna(subset=['total_tests']).copy()

In [ ]:
dfOwidLatest = dfOwidTests.sort_values(by='date').groupby('iso_code').last()

In [ ]:
dfOwidLatest.total_tests.sort_values(ascending=False)

In [ ]:
isUSA = dfOwidLatest.index.isin(['USA'])
NonUSTests = dfOwidLatest[~isUSA].total_tests.sum()
USTests = dfOwidLatest[isUSA].total_tests.sum()

print(f"Total testing done outside of USA: {NonUSTests:.0f}")
print(f"Total tests done in USA : {USTests:.0f}")
print(f"How many more times has the world tested more than the US: {NonUSTests/USTests:.1f}")

In [ ]:
dfOwidLatest[isUSA].sum()

In [ ]:
dfOwidUSA = dfOwid.loc[dfOwid.iso_code=='USA']

dfOwidUSA.set_index('date')[['new_cases','new_deaths']][-60:].plot.line(figsize=(14,6))

In [ ]:
state = 'OK'
dfRollingWeekState = dfCovidtracking.loc[dfCovidtracking.state==state]\
.rolling(window=7).mean()


In [ ]:
plt = dfRollingWeekState.positiveIncrease.plot.line()

In [ ]:
dfJHUData[dfJHUData.state=='Florida'].confirmed.sum()

In [ ]:
doWGA = dfCovidtracking[dfCovidtracking.state=='GA'][['deathIncrease', 'hospitalizedIncrease', 'negativeIncrease','positiveIncrease','totalTestResultsIncrease']]\
.groupby(lambda x: x.dayofweek)

In [ ]:
doWGA.sum()

# Socrata (San Francisco) Data

In [ ]:
!pip install sodapy

In [ ]:
from sodapy import Socrata

In [ ]:
client = Socrata("data.sfgov.org", "LCSuN5PrqWkGtBe9dULkS75Rz")
results_df = pd.DataFrame.from_records(client.get("tvq9-ec9w"))

In [ ]:
res = results_df.astype({'case_count':'int16'}).rename(columns={'specimen_collection_date':'date'})


In [ ]:
dfresConf = res[res.case_disposition=='Confirmed']
dfresDead = res[res.case_disposition=='Death']

In [ ]:
dfTotals = dfresConf.groupby(by='date').sum().join(dfresDead.groupby(by='date').sum(), \
        rsuffix='dead').fillna(0) \
        .rename(columns={'case_count':'positive', 'case_countdead':'dead'})

In [ ]:
chart = dfTotals[['dead', 'positive']][-30:]\
.plot.bar(figsize=(18,6),stacked=True, color=['red','blue'])

In [ ]:
dfresConf.groupby('date').sum().rename(columns={'case_count':'new_cases'}).tail(14)

In [ ]:
dfresConf.groupby('date').sum().rolling(window=7).mean()[-45:].plot.bar(figsize=(20,6))

In [ ]:
dfCovidtracking.loc[dfCovidtracking.state=='CA'][-60::-1].inIcuCurrently.diff().plot.line()

# Marin Data

In [ ]:
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vSRmO2pStf7Ft1yYjDULluZXzUAczGK_B34hDemDBadGEY0HTLwlcadF2cHNEwmmvLetUTf-D2ruYCT/pub?gid=0&single=true&output=csv"

df = pd.read_csv(url, header=1, parse_dates=[0], usecols=[0,1,3,5,6,7], index_col=[0])

In [ ]:
df.tail()

In [ ]:
df[['deaths','positive','tested']].diff()[-14:]

In [ ]:
print(df[['deaths','positive','tested']].diff()[-14:].mean())
print(df[['deaths','positive','tested']].diff()[-14:].std())

In [ ]:
chart= df[['deaths','positive']].diff()[-60:]\
.plot.bar(stacked=True,figsize=(18,5), color=['r','b'])

In [ ]:
chart= df[['positive','tested']].diff()[-30:]\
.plot.bar(stacked=True,figsize=(18,5), color=['darkblue','g'])

In [ ]:
(df.positive.diff()/df.tested.diff()).rolling(window=7).mean()[-14:]

In [ ]:
def pos_test_ratio(depth):
  return df.positive.diff()[-depth:]/df.tested.diff()[-depth:]

df.positive.diff()[-2:]/df.tested.diff()[-2:]

pos_test_ratio(7)


In [ ]:
42/1090


In [ ]:
chart= df[['deaths','positive']].diff()\
.rolling(window=7).mean()[-90:]\
.plot.bar(stacked=True,figsize=(18,5), color=['r','b'])

In [ ]:
marin_pop = 259725

print(f"Marin Population Positive: {(df[-1:].positive.values[0]/marin_pop)*100:2.2f}%")
print(f"Marin Population Tested: {(df[-1:].tested.values[0]/marin_pop)*100:2.2f}%")

print(f"Postive to test ratio: {(df[-1:].positive/df[-1:].tested)[0]*100:2.2f}%")

In [ ]:
df.positive.diff().rolling(window=7).mean().tail()

In [ ]:
d= dfCovidtracking[dfCovidtracking.state=='CA'].head(28)[::-1].positive.diff().max()
print(f"Currrent Record: {d:2.0f}, {hex(int(d))}")
